# <B> Setup for MLOps pipeline with code-pipeline </B>
* Container: conda_python3

## AutoReload

In [1]:
%load_ext autoreload
%autoreload 2

## 1. parameter store 셋팅

In [2]:
import boto3
from utils.ssm import parameter_store

In [3]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
strPrefix = pm.get_params(key="PREFIX")
strPrefix

'SM-MLOPS-PIPELINE-'

## 2. Set code-pipeLine

In [4]:
import os
import sagemaker
from os import path
from utils.code_pipeline import code_pipeline_handler

In [5]:
cph = code_pipeline_handler()

* args for codecommit, build and pipeline

In [6]:
strRepoName = strPrefix + "MLOPS"
strReopDesc = "DEMO - MLOps with code-pipeline"
strSrcDir = os.path.join(os.getcwd(), "./mlops/")

In [7]:
strRepoName

'SM-MLOPS-PIPELINE-MLOPS'

### 2.1 code-commit

In [8]:
## 최초 1회만!!!
! rm -rf .git/

In [9]:
strCloneURL = cph.create_repository(strRepoName, strReopDesc)
cph.clone_from_url(strSrcDir, strCloneURL)
cph.initial_commit_push(strSrcDir)
#cph.delete_repository(strRepoName)

== CREATE REPO ==
  Repository name [SM-MLOPS-PIPELINE-MLOPS] was successfully created!!


Cloning into 'tmp'...




/home/ec2-user/SageMaker/sagemaker-train-deploy
/home/ec2-user/SageMaker/sagemaker-train-deploy/mlops

[master (root-commit) d9135d1] Initial commit
 Committer: EC2 Default User <ec2-user@ip-172-16-94-199.ec2.internal>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly:

    git config --global user.name "Your Name"
    git config --global user.email you@example.com

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 22 files changed, 5407 insertions(+)
 create mode 100644 ReadMe.md
 create mode 100644 codebuild-buildspec.yml
 create mode 100644 pipeline/Untitled.ipynb
 create mode 100644 pipeline/config/__pycache__/config.cpython-37.pyc
 create mode 100644 pipeline/config/config.ini
 create mode 100644 pipeline/config/config.py
 create mode 100644 pipeline/mlops_pipeline.py
 create mode 1

remote: Validating objects: 100%        
To https://git-codecommit.us-east-1.amazonaws.com/v1/repos/SM-MLOPS-PIPELINE-MLOPS
 * [new branch]      master -> master


### 2.2 CodeBuild

In [10]:
strCodeBuildPJTName="demo-mlops-build"

In [11]:
cph.create_build_project(
    strCodeBuildPJTName=strCodeBuildPJTName,
    strAccountId=pm.get_params(key="".join([strPrefix, "ACCOUNT-ID"])),
    strBuildServiceRoleARN=pm.get_params(key="".join([strPrefix, "CODE-BUILD-ROLE-ARN"])),
    strRegionName=pm.get_params(key="".join([strPrefix, "REGION"])),
    strBucketName=pm.get_params(key="".join([strPrefix, "BUCKET"]))
)
#!aws codebuild delete-project --name "demo-mlops-build"

Argments for CodeBuild below:
{'artifacts': {'name': 'demo-mlops-build', 'type': 'CODEPIPELINE'},
 'badgeEnabled': False,
 'cache': {'type': 'NO_CACHE'},
 'description': 'Hello Build Project',
 'environment': {'computeType': 'BUILD_GENERAL1_SMALL',
                 'environmentVariables': [{'name': 'IMAGE_REPO_NAME',
                                           'type': 'PLAINTEXT',
                                           'value': 'iris-model'},
                                          {'name': 'IMAGE_TAG',
                                           'type': 'PLAINTEXT',
                                           'value': 'latest'},
                                          {'name': 'AWS_ACCOUNT_ID',
                                           'type': 'PLAINTEXT',
                                           'value': '419974056037'},
                                          {'name': 'AWS_DEFAULT_REGION',
                                           'type': 'PLAINTEXT',
                    

### 2.3 Edit buildspec.yml
* dependency, excution command, argument 등 수정


In [12]:
!pygmentize mlops/codebuild-buildspec.yml

version: 0.2
phases:
  install:
    runtime-versions:
      python: 3.8
    commands:
      - pip3 install --upgrade --force-reinstall awscli sagemaker
      
  build:
    commands:
      - set -e
      - echo "Starting SageMaker Pipeline for Train Execution"
      - export PYTHONUNBUFFERED=TRUE
      - pushd pipeline
      - python3 mlops_pipeline.py
      - echo "Create/Update of the SageMaker Pipeline and execution completed."
      #- popd

artifacts:
  files:
    - '**/*'


### 2.3. CodePipeline

git push 할 때 파이프라인을 실행하고 싶을 경우 code_pipeline.json에서 PollForSourceChanges 를 True로 변경하면 된다. 

In [13]:
strCodePipelineName = "demo-mlops-code-pipeline"

In [14]:
#cph = code_pipeline_handler()
cph.create_execute_code_pipeline(
    strCodePipelineName=strCodePipelineName,
    strPipelineRoleARN=pm.get_params(key="".join([strPrefix, "CODE-PIPELINE-ROLE-ARN"])),
    strRegionName=pm.get_params(key="".join([strPrefix, "REGION"])),
    strBucketName=pm.get_params(key="".join([strPrefix, "BUCKET"])),
    strRepoName=strRepoName,
    strCodeBuildPJTName=strCodeBuildPJTName,
)
#!aws codepipeline delete-pipeline --name "demo-mlops-code-pipeline"

Couldn't find Pipeline: [demo-mlops-code-pipeline], so, create new pipeline.
Create CodePipeline
Argments for CodeBuild below:
{'pipeline': {'artifactStore': {'location': 'sagemaker-us-east-1-419974056037',
                                'type': 'S3'},
              'name': 'demo-mlops-code-pipeline',
              'roleArn': 'arn:aws:iam::419974056037:role/SM-MLOPS-PIPELINE-CodePipelineRole',
              'stages': [{'actions': [{'actionTypeId': {'category': 'Source',
                                                        'owner': 'AWS',
                                                        'provider': 'CodeCommit',
                                                        'version': '1'},
                                       'configuration': {'BranchName': 'master',
                                                         'OutputArtifactFormat': 'CODE_ZIP',
                                                         'PollForSourceChanges': 'false',
                                 

* restart

In [16]:
cph.start_pipeline_execution(strCodePipelineName)

Start pipeline execution for [demo-mlops-code-pipeline]
